# Binding detailed example: Minuit2

Let's try a non-trivial example: Minuit2 (6.14.0 standalone edition)

### Requirements

* Minuit2 standalone edition (included)

### Expectations
* Be able to minimize a very simple function and get some parameters

# Step 1: Get source

* Download Minuit2 source (already provided in `minuit2`)

# Step 2: Plan interface

* You should know what the C++ looks like, and know what you want the Python to look like
* For now, let's replicate the C++ experience

For example: a simple minimizer for $f(x) = x^2$ (should quickly find 0 as minimum):

* Define FCN
* Setup parameters
* Minimize
* Print result

Will use print out for illustration (instead of `MnPrint::SetLevel`)

In [ ]:
%%writefile SimpleFCN.h
#pragma once

#include <Minuit2/FCNBase.h>
#include <Minuit2/FunctionMinimum.h>
#include <Minuit2/MnPrint.h>
#include <Minuit2/MnMigrad.h>

using namespace ROOT::Minuit2;

class SimpleFCN : public FCNBase {
    // Always 0.5 for these sorts of fits
    
    double Up() const override {return 0.5;}
    
    // This computes whatever you are going to minimize
    double operator()(const std::vector<double> &v) const override {
        std::cout << "val = " << v.at(0) << std::endl;
        return v.at(0)*v.at(0);
    }
};

In [ ]:
%%writefile simpleminuit.cpp
#include "SimpleFCN.h"

int main() {
    SimpleFCN fcn;
    MnUserParameters upar;
    upar.Add("x", 1., 0.1);
    MnMigrad migrad(fcn, upar);
    FunctionMinimum min = migrad();
    std::cout << min << std::endl;
}

In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.4)
project(Minuit2SimpleExamle LANGUAGES CXX)

add_subdirectory(minuit2)

# Temporary patch, only needed in some versions of Minuit2
target_compile_definitions(Minuit2Math PUBLIC MATHCORE_STANDALONE)

add_executable(simpleminuit simpleminuit.cpp SimpleFCN.h)
target_link_libraries(simpleminuit PRIVATE Minuit2::Minuit2)

Standard CMake configure and build (using Ninja instead of Make for speed)

In [ ]:
!cmake -GNinja .
!cmake --build .

In [ ]:
!./simpleminuit

## Step 3: Bind parts we need

* subclassable FCNBase
* MnUserParameters (constructor and `Add(string, double, double)`)
* MnMigrad (constructor and operator())
* FunctionMinimum (cout)


In [ ]:
%mkdir pyminuit2

## Recommended structure of a Pybind11 program

#### main.cpp

* Builds module
* Avoids imports (fast compile)

```cpp
include <pybind11/pybind11.h>
namespace py = pybind11;

void init_part1(py::module &);
void init_part2(py::module &);

PYBIND11_MODULE(mymodule, m) {
    m.doc() = "Real code would never have such poor documentation...";
    init_part1(m);
    init_part2(m);
}
```

In [ ]:
%%writefile pyminuit2/pyminuit2.cpp

#include <pybind11/pybind11.h>

namespace py = pybind11;

void init_FCNBase(py::module &);
void init_MnUserParameters(py::module &);
void init_MnMigrad(py::module &);
void init_FunctionMinimum(py::module &);

PYBIND11_MODULE(minuit2, m) {
    init_FCNBase(m);
    init_MnUserParameters(m);
    init_MnMigrad(m);
    init_FunctionMinimum(m);
}

We will put all headers in a collective header (not a good idea unless you are trying to show files one per slide).

In [ ]:
%%writefile pyminuit2/PyHeader.h
#pragma once
#include <pybind11/pybind11.h>
#include <pybind11/functional.h>
#include <pybind11/numpy.h>
#include <pybind11/stl.h>

#include <Minuit2/FCNBase.h>
#include <Minuit2/MnMigrad.h>
#include <Minuit2/MnApplication.h>
#include <Minuit2/MnUserParameters.h>
#include <Minuit2/FunctionMinimum.h>

namespace py = pybind11;
using namespace pybind11::literals;
using namespace ROOT::Minuit2;

## Overloads

* Pure virtual methods cannot be instantiated in C++
* Have to provide "Trampoline class" to provide Python class

In [ ]:
%%writefile pyminuit2/FCNBase.cpp
#include "PyHeader.h"
class PyFCNBase : public FCNBase {
   public:
     using FCNBase::FCNBase;

     double operator()(const std::vector<double> &v) const override {
         PYBIND11_OVERLOAD_PURE_NAME(
             double, FCNBase, "__call__", operator(), v);}

     double Up() const override {
         PYBIND11_OVERLOAD_PURE(double, FCNBase, Up, );}
 };
void init_FCNBase(py::module &m) {
    py::class_<FCNBase, PyFCNBase>(m, "FCNBase")
         .def(py::init<>())
         .def("__call__", &FCNBase::operator())
         .def("Up", &FCNBase::Up);
}

## Overloaded function signatures:
* C++11 syntax: `(bool (MnUserParameters::*)(const std::string &, double)) &MnUserParameters::Add`
* C++14 syntax: `py::overload_cast<const std::string &, double>(&MnUserParameters::Add)`

In [ ]:
%%writefile pyminuit2/MnUserParameters.cpp
#include "PyHeader.h"

void init_MnUserParameters(py::module &m) {
    py::class_<MnUserParameters>(m, "MnUserParameters")
        .def(py::init<>())
        .def("Add", (bool (MnUserParameters::*)(const std::string &, double)) &MnUserParameters::Add)
        .def("Add", (bool (MnUserParameters::*)(const std::string &, double, double)) &MnUserParameters::Add)
    ;
}

## Adding default arguments (and named arguments)
* Using `""_a` literal, names and even defaults can be added 

In [ ]:
%%writefile pyminuit2/MnMigrad.cpp
#include "PyHeader.h"
        
void init_MnMigrad(py::module &m) {
    py::class_<MnApplication>(m, "MnApplication")
        .def("__call__",
             &MnApplication::operator(),
             "Minimize the function, returns a function minimum",
             py::arg("maxfcn")    = 0,
             "tolerance"_a = 0.1);
    
    py::class_<MnMigrad, MnApplication>(m, "MnMigrad")
        .def(py::init<const FCNBase &, const MnUserParameters &, unsigned int>(),
             "fcn"_a, "par"_a, "stra"_a = 1)
    ;
}    

## Lambda functions

* Pybind11 accepts lambda functions, as well

In [ ]:
%%writefile pyminuit2/FunctionMinimum.cpp
#include "PyHeader.h"

#include <sstream>
#include <Minuit2/MnPrint.h>

void init_FunctionMinimum(py::module &m) {
    py::class_<FunctionMinimum>(m, "FunctionMinimum")
        .def("__str__", [](const FunctionMinimum &self) {
            std::stringstream os;
            os << self;
            return os.str();
        })
    ;
}

In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.4)
project(Minuit2SimpleExamle LANGUAGES CXX)

set(CMAKE_POSITION_INDEPENDENT_CODE ON)
add_subdirectory(minuit2)
add_executable(simpleminuit simpleminuit.cpp SimpleFCN.h)
target_link_libraries(simpleminuit PRIVATE Minuit2::Minuit2)

target_compile_definitions(Minuit2Math PUBLIC MATHCORE_STANDALONE)

find_package(pybind11)

file(GLOB OUTPUT pyminuit2/*.cpp)
pybind11_add_module(minuit2 ${OUTPUT})
target_link_libraries(minuit2 PUBLIC Minuit2::Minuit2)

In [ ]:
!cmake .
!cmake --build .

# Usage

We can now use our module! (Built in the current directory by CMake)

In [ ]:
import minuit2

class SimpleFCN (minuit2.FCNBase):
    def Up(self):
        return 0.5
    def __call__(self, v):
        print("val =", v[0])
        return v[0]**2

In [ ]:
fcn = SimpleFCN()
upar = minuit2.MnUserParameters()
upar.Add("x", 1., 0.1)
migrad = minuit2.MnMigrad(fcn, upar)
min = migrad()

In [ ]:
print(min)

# Done

* See [GooFit's built in Minuit2 bindings](https://github.com/GooFit/GooFit/tree/master/python/Minuit2) for a more complete example
* Pybind11 bindings can talk to each other at the C level!

# Bonus:

This is the setup.py file for the Miniut2 bindings. With this, you can use the standard Python tools to build! (but slower and more verbose than CMake)

In [ ]:
%%writefile pyproject.toml

[build-system]
requires = [
    "setuptools>=42",
    "wheel",
    "pybind11>=2.6"
]

build-backend = "setuptools.build_meta"


The pyproject.toml will be picked up by a proper pip install. We already have pybind11 and don't need to worry about it, so we'll just keep using build_ext. Don't do this in real development unless you know what you are doing.

In [ ]:
%%writefile setup.py

from setuptools import setup
from pybind11.setup_helpers import Pybind11Extension, ParallelCompile
from pathlib import Path

sources = set(str(p) for p in Path('minuit2/src').glob('**/*.cxx'))
sources.remove('minuit2/src/TMinuit2TraceObject.cxx')

## Add your sources to `sources`
sources |= set(str(p) for p in Path('pyminuit2').glob('*.cpp'))

ext_modules = [
    Pybind11Extension(
        'minuit2',
        sorted(sources),
        include_dirs=['minuit2/inc'],
        cxx_std=11,
        define_macros=[('WARNINGMSG', None),
                       ('MATH_NO_PLUGIN_MANAGER', None),
                       ('ROOT_Math_VecTypes', None),
                       ('MATHCORE_STANDALONE', None)
                      ],
    ),
]

ParallelCompile().install()

setup(
    name='minuit2',
    version='6.18.0',
    author='Henry Schriener',
    author_email='hschrein@cern.ch',
    url='https://github.com/GooFit/Minuit2',
    description='A Pybind11 Minuit2 binding',
    long_description='',
    ext_modules=ext_modules,
    install_requires=['numpy>=1.10'],
    zip_safe=False,
)

In [ ]:
!python setup.py build_ext --inplace